In [83]:
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error,mean_absolute_error
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit



In [84]:
# load dataset
# file_list=['BTCUSD-1h','BTCUSD-all','BTCUSD-N2Y','BTCUSD-N4Y',
#            'ETHUSD-1h','ETHUSD-all','ETHUSD-N2Y','ETHUSD-N4Y',           
#            'USDTUSD-1h','USDTUSD-all','USDTUSD-N2Y','USDTUSD-N4Y',  
#            'BNBUSD-1h','BNBUSD-all','BNBUSD-N2Y','BNBUSD-N4Y',
#            'USDCUSD-1h','USDCUSD-all','USDCUSD-N2Y','USDCUSD-N4Y'         
#            ]

# spilt_list=[0.3,0.2,0.1]

file_list=['BTCUSD-all']
spilt_list=[0.3]

files_list=list()
spilts_list=list()
Classifiers_list=list()

train_mae=list()
train_rmse=list()
test_mae=list()
test_rmse=list()


In [87]:
for file in file_list:
    for splits in spilt_list:
        files_list.append(file)
        spilts_list.append(splits)

        data = pd.read_csv(f'../Data/{file}.csv') 
        train_data, test_data = train_test_split(data['Close'], test_size=splits, shuffle=False)

        # Take the logarithm
        log_train = np.log(train_data)
      

        # Calculate mean and standard deviation
        mean = np.mean(log_train)
        std = np.std(log_train)*3

        # Z-score normalization
        standardized_train = (log_train - mean) / std


        tscv = TimeSeriesSplit(n_splits=5)
        best_model = None
        best_rmse = np.inf

        for train_index, val_index in tscv.split(standardized_train):
            train_val_data, val_data = standardized_train[train_index], standardized_train[val_index]

            
            model = auto_arima(train_val_data,max_p=20,max_q=20,seasonal=False)
            
            
            forecast = model.predict(len(val_data))


            rmse = np.sqrt(mean_squared_error(val_data, forecast))
            if rmse < best_rmse:
                best_rmse = rmse
                best_model = model


        best_model.fit(standardized_train,disp=True)    

        print(f'{file} {splits}')
        best_model_params = best_model.get_params()['order']
        print("最佳模型參數:ARIMA", best_model_params)
        Classifiers_list.append(f'ARIMA{best_model_params}')
        
        # forecast

        forecast = best_model.predict(len(test_data))       
        original_predict_test = np.exp(forecast*std+mean)

        test_data_mae=mean_absolute_error(test_data,original_predict_test)
        test_data_rmse=mean_squared_error(test_data,original_predict_test)**0.5        
        test_mae.append(test_data_mae)
        test_rmse.append(test_data_rmse)       

        # predict train
        train_forecast = best_model.predict_in_sample()      
        original_predict_train = np.exp(train_forecast*std+mean)

        train_data_mae=mean_absolute_error(train_data,original_predict_train)
        train_data_rmse=mean_squared_error(train_data,original_predict_train)**0.5
        train_mae.append(train_data_mae)
        train_rmse.append(train_data_rmse) 

        print(f'MAE train:{train_data_mae:.4f} test:{test_data_mae:.4f}')   
        print(f'RMSE train:{train_data_rmse:.4f} test:{test_data_rmse:.4f}')         
        
        

BTCUSD-all 0.3
最佳模型參數:ARIMA (0, 1, 0)


ValueError: Found input variables with inconsistent numbers of samples: [2332, 1944]

In [82]:
Summary={'Data':files_list,'Classifier':Classifiers_list,'Test Split':spilts_list,
            'train_mae':train_mae,          
            'train_rmse':train_rmse,          
            'test_mae':test_mae,           
            'test_rmse':test_rmse            
         }

df_Summary = pd.DataFrame(Summary)
df_Summary.to_excel("Summary-AutoARIMA-14m.xlsx",index=False)  
df_Summary

,Data,Classifier,Test Split,train_mae,train_rmse,test_mae,test_rmse
0,BTCUSD-1h,"ARIMA(1, 0, 0)",0.3,63.633445,111.894121,4340.710340,4578.912755
1,BTCUSD-1h,"ARIMA(1, 0, 0)",0.2,82.171014,187.699847,298.804426,355.613359
2,BTCUSD-1h,"ARIMA(2, 0, 1)",0.1,82.857908,175.264443,454.554422,509.544043
3,BTCUSD-all,"ARIMA(0, 1, 0)",0.3,159.296820,377.283091,79562.905872,102760.626694
4,BTCUSD-all,"ARIMA(0, 1, 0)",0.2,317.014105,762.689941,62088.337217,70129.006749
5,BTCUSD-all,"ARIMA(0, 1, 0)",0.1,359.594909,802.328856,5037.032061,5780.397107
6,BTCUSD-N2Y,"ARIMA(0, 1, 0)",0.3,376.620153,818.239768,100143.685524,123502.437561
7,BTCUSD-N2Y,"ARIMA(2, 1, 0)",0.2,473.944836,950.408380,24421.949812,26675.055344
8,BTCUSD-N2Y,"ARIMA(0, 1, 0)",0.1,469.299094,927.721928,2800.030263,3204.246691
9,BTCUSD-N4Y,"ARIMA(0, 1, 0)",0.3,615.364782,1140.585707,14888.525408,15786.237633


In [79]:
data = pd.read_csv('../Data/BTCUSD-all.csv') 

train_data, test_data = train_test_split(data['Close'], test_size=0.3, shuffle=False)
model = auto_arima(train_data, seasonal=False)

n_periods = len(test_data)
forecast = model.predict(n_periods=n_periods)
rmse = mean_squared_error(test_data, forecast, squared=False)
print("測試集(RMSE):", rmse)


train_forecast = model.predict_in_sample()
train_rmse = mean_squared_error(train_data, train_forecast, squared=False)
print("訓練集(RMSE):", train_rmse)


測試集(RMSE): 107183.31430396551
訓練集(RMSE): 372.9703178875585


In [81]:
print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                      y   No. Observations:                 2332
Model:               SARIMAX(4, 2, 2)   Log Likelihood              -17105.089
Date:                Mon, 20 Nov 2023   AIC                          34224.179
Time:                        13:36:18   BIC                          34264.454
Sample:                             0   HQIC                         34238.853
                               - 2332                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.7927      0.052    -15.140      0.000      -0.895      -0.690
ar.L2          0.0018      0.008      0.230      0.818      -0.014       0.017
ar.L3         -0.1061      0.009    -11.293      0.0

In [14]:
best_model_params = model.get_params()['order']
best_model_params

(4, 2, 2)